In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd /content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1

/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1


In [4]:
#importing the necessary libraries
import tensorflow as tf
from tensorflow.keras import layers
from skimage.io import imshow
from pathlib import Path
import pandas as pd

# Input data files are available in the "../mura_unzipped/" directory.
# Running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1"))


['valid', 'train', 'Copy of train_image_paths.csv', 'Copy of valid_image_paths.csv', 'train_image_paths1.csv', 'valid_image_paths1.csv', '.ipynb_checkpoints', 'bonemodel.h5', 'train_labeled_studies.csv', 'valid_labeled_studies.csv', 'train_image_paths.csv', 'valid_image_paths.csv', 'bonemodel1.h5', 'DEMO DATA 1', 'Copy of data_preprocessing.ipynb']


In [5]:
dataset_root = Path('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1')

In [6]:
list(dataset_root.iterdir())

[PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/valid'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/train'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/Copy of train_image_paths.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/Copy of valid_image_paths.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/train_image_paths1.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/valid_image_paths1.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/.ipynb_checkpoints'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/bonemodel.h5'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/train_labeled_studies.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1/valid_labeled_studies.csv'),
 PosixPa

In [7]:
#df is the generated dataframe whose head can be visualized below
df = pd.read_csv('train_image_paths.csv', header=None, names=['filename'])
df.head()

,filename
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...


In [8]:
#Adding class column to the dataframe with positive or negative label
#Classification is done on basis of the image names (postive/negative)
df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
df.head()

,filename,class
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive


In [9]:
#A function to generate the dataframe for a csv file
def generate_df(dataset_root, csv_name):
    df = pd.read_csv(dataset_root/csv_name, header=None, names=['filename'])
    df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
    return df

In [10]:
list(dataset_root.parent.iterdir())

[PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/Bonenormalabnormalclassification'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/train_labeled_studies.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/train_image_paths.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/valid_labeled_studies.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/valid_image_paths.csv'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1.zip'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/Bone Data'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA-v1.1'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/boneclassification.ipynb'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/MURA(DenseNet) .ipynb'),
 PosixPath('/content/gdrive/MyDrive/Colab Notebooks/Bone fracture/bonedoctor-master'),
 PosixPath('/

In [11]:
#Dividing the image data generated into train set and validation set
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255)
#Creating training set
train_gen = datagen.flow_from_dataframe(generate_df(dataset_root, 'train_image_paths1.csv'),
                                        directory=dataset_root.parent,
                                        target_size=(224, 224),
                                        class_mode='binary')
#Creating validation set
valid_gen = datagen.flow_from_dataframe(generate_df(dataset_root, 'valid_image_paths1.csv'),
                                        directory=dataset_root.parent,
                                        target_size=(224, 224),
                                        class_mode='binary')

Found 20 validated image filenames belonging to 2 classes.
Found 20 validated image filenames belonging to 2 classes.


In [12]:
#Downloading the densenet model pretrained on the imagenet dataset
densenet = tf.keras.applications.DenseNet169(weights='imagenet', include_top = False, input_shape=(224, 224, 3))

51888128/51877672 [==============================] - 3s 0us/step


In [13]:
#Freezing the weights of the pretrained model
densenet.trainable = False

In [14]:
densenet.summary()

Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                        

In [15]:
#Adding the Flatten layer and the sigmoid classification layer to the pretrained densenet model
model = tf.keras.models.Sequential([
    densenet,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 dense (Dense)               (None, 1)                 81537     
                                                                 
Total params: 12,724,417
Trainable params: 81,537
Non-trainable params: 12,642,880
_________________________________________________________________


In [17]:
#Compiling the model using adam optimizer
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [18]:
#Fitting the model on training test
model.fit_generator(train_gen, epochs=5, validation_data=valid_gen, use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
1/1 [==============================] - 57s 57s/step - loss: 0.7255 - accuracy: 0.6500 - val_loss: 0.9405 - val_accuracy: 0.6000
Epoch 2/5
1/1 [==============================] - 1s 1s/step - loss: 7.5309e-04 - accuracy: 1.0000 - val_loss: 1.0561 - val_accuracy: 0.7000
Epoch 3/5
1/1 [==============================] - 1s 1s/step - loss: 1.7897e-05 - accuracy: 1.0000 - val_loss: 1.2089 - val_accuracy: 0.7500
Epoch 4/5
1/1 [==============================] - 1s 1s/step - loss: 1.1958e-06 - accuracy: 1.0000 - val_loss: 1.3599 - val_accuracy: 0.7500
Epoch 5/5
1/1 [==============================] - 1s 1s/step - loss: 1.3336e-07 - accuracy: 1.0000 - val_loss: 1.4997 - val_accuracy: 0.7500


In [19]:
#Saving the model weights
model.save("bonemodel.h5")
print("Saved model to disk")

Saved model to disk


In [20]:
# Loading the saved model

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
model_new = load_model('bonemodel.h5')
model_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 7, 7, 1664)        12642880  
                                                                 
 flatten (Flatten)           (None, 81536)             0         
                                                                 
 dense (Dense)               (None, 1)                 81537     
                                                                 
Total params: 12,724,417
Trainable params: 81,537
Non-trainable params: 12,642,880
_________________________________________________________________


In [23]:
# precision tp / (tp + fp)
from sklearn.metrics import precision_score
precision = precision_score(train_gen, valid_gen)
print('Precision: %f' % precision)
    


ValueError: ignored

In [ ]:
    # recall: tp / (tp + fn)
    recall = recall_score(y_true, y_pred_binary)
    print('Recall: %f' % recall)